In [1]:
import skimage.io
import skimage.filters
import sys

import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
import os

from PIL import Image, ImageFilter
import ImageTools

In [349]:
def stack_xy(path2 = "./source/", dest='output_leaf/img_to_architecture/square/', ress=96, ressfinal=1024, thicken=0, rect=0, destfile=None): 
 names=[]
 density=[]
 seeds=[]
 dirs = os.listdir(path2)

 isExist = os.path.exists(dest)

 if not isExist:
  
  # Create a new directory because it does not exist 
  os.makedirs(dest)
  print("The new directory is created: ", dest)
 print("Initial res: ", ress, "final max res: ", ressfinal)

 count=-1
 for i, item in enumerate(dirs):
        fullpath = os.path.join(path2,item)         #corrected
        if os.path.isfile(fullpath):
 
            #print (fullpath, i)          
            
            image = cv2.imread(fullpath)
            
            img_gray_T=cv2.resize(image, (ress,ress), interpolation = cv2.INTER_AREA)
            
            if i%16==0: #create initial array if starting off
                count+=1
                if count==1:
                    img_full=img_gray
                if count>1:
                    img_full=np.concatenate((img_full, img_gray), axis=1)
                img_gray=cv2.resize(image, (ress,ress), interpolation = cv2.INTER_AREA)
            else:
                img_gray = np.concatenate((img_gray, img_gray_T), axis=0)
                if i==255:
                    img_full=np.concatenate((img_full, img_gray), axis=1)
            
            
 if thicken>0:
                print ("Thickening resulting structure using dilation...iterations: ", thicken)
                kernel = np.ones((2,5),np.uint8)
                img_full = cv2.dilate(img_full,kernel,iterations = thicken)           
 if rect>0:
                #add rectangle - for better 3D processing
                #rcolor = (255, 255, 255) #black
                rcolor = (0, 0, 0) #white
                cv2.rectangle(img_full, (0,0), (img_full.shape[1], img_full.shape[0]), rcolor, thickness=10)
            
            
 #resize to final desired dimension...

 #current res:
 resx=img_full.shape[0]
 resy=img_full.shape[1]
    
 minres=min (resx, resy)
 maxres=max (resx, resy)
 print ("Largest res in final image: ", maxres, "... but should be: ", ressfinal, "... ratio: ", ressfinal/maxres)
    
 img_full=cv2.resize(img_full, (int(ressfinal/maxres*resy),int(ressfinal/maxres*resx)), interpolation = cv2.INTER_AREA)
            
 
 fnamee=dest +'%4.4d'%i+ '_smoothed.png'
 if destfile != None:
    fnamee=dest +'%4.4d'%destfile+ '_smoothed.png'
    print ("Writing file name with provided number: ",fnamee )
 cv2.imwrite(fnamee, img_full)

 return  

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [313]:
filename='corner_grad.png'
label='corner_grad'
csv_label="./img_to_architecture_"+label+".csv"
outdir_label="./output_leaf/img_to_architecture/"+label+"/"

In [323]:
image_file = Image.open(filename)

if len(np.asarray(image_file).shape)==2:
    gray = image_file.resize((16,16))
elif np.asarray(image_file).shape[2]==4:
    image_file.load()
    rgb = Image.new("RGB", image_file.size, (255, 255, 255))
    rgb.paste(image_file, mask = image_file.split()[3])
    gray=rgb.convert("L").resize((16,16))
elif np.asarray(image_file).shape[2]==3:
    gray = image_file.convert("L").resize((16,16))
elif np.asarray(image_file).shape[2]==1:
    gray = image_file.resize((16,16))

In [324]:
gray = gray.transpose(Image.FLIP_LEFT_RIGHT)
threshold = np.asarray(gray).mean()
bw = gray.point(lambda p: p > threshold and 255)  
bw.save("bw.png", format="png")
pixels_bw = np.asarray(bw)

In [ ]:
#blur = bw.filter(ImageFilter.BoxBlur(1))
#np.asarray(blur)

gaussian = bw.filter(ImageFilter.GaussianBlur(1))
np.asarray(gaussian)
gaussian.save("gaussian.png", format="png")
#gaussian.show()

normalized = NormalizeData(np.asarray(gaussian))

In [307]:
csv_points=np.stack((normalized.reshape((256,)), np.zeros(256)), axis=1)
pd.DataFrame(csv_points).to_csv(csv_label, header=['x','y'], index=False)

In [308]:
#generate images
!python generate.py --outdir=./output_leaf/img_to_architecture/square/ --trunc=.7 --frames=1 --random_seed=0 --process="interpolation" --interpolation="2Dpoint_CSV" --csvfile=./img_to_architecture_square.csv --seeds=441,863,68   --network=leaf-network-snapshot.pkl 

#process into unit cells
path = "output_leaf/img_to_architecture/"+label+"/frames/" 
dest="output_leaf/img_to_architecture/"+label+"/processed/"

names, seeds, density=ImageTools.remove_small(path, dest, 30000, 1024,mirrorcopy=1, ressfinal=256, filt1=32, thicken=0, rect=0)

#stack
path="./output_leaf/img_to_architecture/"+label+"/processed/"
dest="./output_leaf/img_to_architecture/"+label+"/stack/"
stack_xy(path2 = path, dest=dest, ress=96, ressfinal=4096, thicken=2, rect=0, destfile=None)

Loading networks from "leaf-network-snapshot.pkl"...
Interpolation algorithm...
Generate from CSV file...
2D point generation (read multiple points from CSV file)
Generate from CSV file - read all data:  ./img_to_architecture_square.csv
[[1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.1 (GCC) 20180722
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100


 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.92647059 0.        ]
 [0.92647059 0.        ]
 [0.94607843 0.        ]
 [0.9754902  0.        ]
 [0.99509804 0.        ]
 [0.99509804 0.        ]
 [0.9754902  0.        ]
 [0.94607843 0.        ]

In [354]:
#stack
label='square'
path="./output_leaf/img_to_architecture/"+label+"/processed/"
dest="./output_leaf/img_to_architecture/"+label+"/stack/"
stack_xy(path2 = path, dest=dest, ress=96, ressfinal=512, thicken=2, rect=0, destfile=None)

Initial res:  96 final max res:  512
Thickening resulting structure using dilation...iterations:  2
Largest res in final image:  1536 ... but should be:  512 ... ratio:  0.3333333333333333


## Image directly to arch (no blur)

In [329]:
filename='corner_grad.png'
label='corner_grad'
csv_label="./img_to_architecture_"+label+".csv"
outdir_label="./output_leaf/img_to_architecture/"+label+"/"
image_file = Image.open(filename)

In [330]:
if len(np.asarray(image_file).shape)==2:
    gray = image_file.resize((16,16))
elif np.asarray(image_file).shape[2]==4:
    image_file.load()
    rgb = Image.new("RGB", image_file.size, (255, 255, 255))
    rgb.paste(image_file, mask = image_file.split()[3])
    gray=rgb.convert("L").resize((16,16))
elif np.asarray(image_file).shape[2]==3:
    gray = image_file.convert("L").resize((16,16))
elif np.asarray(image_file).shape[2]==1:
    gray = image_file.resize((16,16))

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [332]:
gray = gray.transpose(Image.FLIP_LEFT_RIGHT)
normalized = NormalizeData(np.asarray(gray))
csv_points=np.stack((normalized.reshape((256,)), np.zeros(256)), axis=1)
pd.DataFrame(csv_points).to_csv(csv_label, header=['x','y'], index=False)

In [335]:
#generate images

!python generate.py --outdir=./output_leaf/img_to_architecture/corner_grad/ --trunc=.7 --frames=1 --random_seed=0 --process="interpolation" --interpolation="2Dpoint_CSV" --csvfile=./img_to_architecture_corner_grad.csv --seeds=441,863,68   --network=leaf-network-snapshot.pkl 

#process into unit cells
path = "output_leaf/img_to_architecture/"+label+"/frames/" 
dest="output_leaf/img_to_architecture/"+label+"/processed/"

names, seeds, density=ImageTools.remove_small(path, dest, 30000, 1024,mirrorcopy=1, ressfinal=256, filt1=32, thicken=0, rect=0)



#stack
path="./output_leaf/img_to_architecture/"+label+"/processed/"
dest="./output_leaf/img_to_architecture/"+label+"/stack/"
stack_xy(path2 = path, dest=dest, ress=64, ressfinal=4096, thicken=2, rect=0, destfile=None)

Loading networks from "leaf-network-snapshot.pkl"...
Interpolation algorithm...
Generate from CSV file...
2D point generation (read multiple points from CSV file)
Generate from CSV file - read all data:  ./img_to_architecture_corner_grad.csv
[[0.28571429 0.        ]
 [0.28571429 0.        ]
 [0.21428571 0.        ]
 [0.21428571 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.07142857 0.        ]
 [0.07142857 0.        ]
 [0.07142857 0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.07142857 0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.28571429 0.        ]
 [0.28571429 0.        ]
 [0.28571429 0.        ]
 [0.21428571 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.14285714 0.        ]
 [0.07142857 0.        ]
 [0.07142857 0.        ]
 [0.07142857 0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.0714

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.1 (GCC) 20180722
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100


        -0.33679202, -0.95313246, -1.11637396,  0.07986203,  0.16475044,
        -0.33352612,  0.27973382]]), array([[-0.11181088,  0.44155229,  0.25717067,  0.43304841, -0.95613507,
        -0.80367382, -0.16907108, -0.66890326, -0.36815725,  1.51243283,
         0.9009927 ,  0.00700806,  0.7791711 ,  0.66058938,  0.07114607,
        -0.88793343, -0.70995084,  0.32177651,  1.62540547, -1.19479701,
         0.91729526,  0.84513826,  0.58582253, -1.41428972,  0.77942167,
        -0.10460932, -0.48873315, -1.32276274,  0.26254502,  0.78723745,
        -0.10581101, -0.64508072, -0.47882108,  0.84875172,  0.25582823,
        -0.71630939, -0.99378442, -1.06560642,  0.8267394 ,  0.47443718,
         0.50491043,  1.58264911,  0.44360756, -1.74632665,  0.33501249,
         1.1352716 ,  1.14434531,  0.21389988,  1.56901242, -1.07244544,
         0.23940208, -0.61291196,  0.51821171,  0.86813896,  0.51587172,
        -0.32335936, -0.85962156,  0.09941209, -0.80665843,  1.09959134,
        -1.67

        -7.00873320e-01, -8.03064583e-01, -1.57860639e-01,
         7.74577387e-01, -3.41888893e-01,  3.45191327e-01,
        -2.06059907e-01,  7.82886965e-01,  8.63432476e-01,
         1.49575691e+00, -6.68013707e-01, -2.62641866e-01,
         4.10219877e-01, -3.74134851e-01, -7.99492359e-01,
        -9.59389852e-01,  1.34375642e-02,  3.82567114e-02,
        -3.16187581e-01,  3.09063897e-01]]), array([[-0.11181088,  0.44155229,  0.25717067,  0.43304841, -0.95613507,
        -0.80367382, -0.16907108, -0.66890326, -0.36815725,  1.51243283,
         0.9009927 ,  0.00700806,  0.7791711 ,  0.66058938,  0.07114607,
        -0.88793343, -0.70995084,  0.32177651,  1.62540547, -1.19479701,
         0.91729526,  0.84513826,  0.58582253, -1.41428972,  0.77942167,
        -0.10460932, -0.48873315, -1.32276274,  0.26254502,  0.78723745,
        -0.10581101, -0.64508072, -0.47882108,  0.84875172,  0.25582823,
        -0.71630939, -0.99378442, -1.06560642,  0.8267394 ,  0.47443718,
         0.504910

         0.79126875, -1.71666853,  0.48294888, -0.24694644,  0.45911504,
        -0.81823694, -0.86749067, -2.05596524, -0.42387623, -0.0113182 ,
        -0.34594723, -0.59951905, -0.5786247 , -0.43693593,  1.06340874,
         0.40041261,  0.2966923 , -1.24489271,  0.67874556,  1.95271831,
        -0.93951591,  0.07498085, -0.51199859,  0.75789576, -0.92148262,
         0.39716291,  1.88901521,  0.97963798,  1.87563004,  1.08051751,
        -2.38325604,  0.39699562, -0.80111168,  0.94051397,  1.06312911,
         0.90552665, -0.06058684, -0.01568607,  1.21212917,  0.91224489,
        -0.63293345, -1.10422611, -0.66829589,  1.11557903, -1.84344145,
        -0.93597601,  0.68619912,  0.51225118, -0.04074965, -0.43948364,
        -0.29055027,  0.85082745, -0.50557407,  0.6968367 ,  0.33327735,
        -0.41510723,  1.05293113, -1.83137069, -0.72406496, -2.16165045,
         0.92972268,  0.35003   ,  1.47609652,  0.29103438, -0.08680252,
         0.51527466, -0.36949395, -0.65169246,  1.5

...

         7.32130210e-02,  1.11285721e-01, -5.14650398e-01,
        -7.63896929e-02,  9.76128064e-01,  5.91388387e-01,
        -2.89471014e-01,  3.69643016e-02,  3.98954532e-01,
         1.16947476e+00,  4.61424450e-01,  5.93357456e-01,
        -4.00907659e-01,  5.31976178e-02, -4.25426342e-01,
        -2.05387364e-01,  8.67877843e-01,  6.41741963e-01,
        -4.23266929e-01,  8.36102604e-02,  7.88319256e-01,
        -2.75823725e-01, -2.68063788e-01, -1.73368005e-01,
         9.99834627e-01,  1.36753150e+00,  4.80978274e-01,
        -1.34631096e+00,  2.06695870e-01,  6.09630678e-01,
        -1.40492984e+00, -2.21659167e-01,  5.74331098e-01,
         9.90711791e-01,  2.32172574e-01,  7.96431482e-01,
        -5.81805911e-01,  4.01143435e-01, -6.78929201e-01,
         1.06339670e+00, -8.98988891e-01,  2.92653021e-01,
        -5.63480138e-01,  2.51291890e-01,  8.22943260e-01,
        -6.77583542e-01,  4.24124828e-01,  7.01599943e-01,
        -7.72835624e-01,  1.22133612e+00,  3.85541039e-0

        -2.32927884e-01,  2.06421815e-01, -2.59731653e-01,
        -7.48804276e-01, -2.96232750e-01, -6.71845982e-01,
         2.92037703e-01, -1.70889024e+00, -3.05754845e-01,
        -1.87765699e+00,  2.24125912e-01,  1.84242425e-01,
         8.08743256e-01,  7.23591674e-01, -6.33069538e-01,
         3.67812583e-01, -2.93233547e-01,  5.72976710e-01,
         1.64561237e+00,  1.41275307e+00,  5.54397607e-02,
         2.71827864e-01, -2.23417046e-01,  5.96246182e-01,
         3.32867498e-01,  1.25959896e+00, -1.25443338e+00,
        -2.70161092e-01, -4.05505140e-02,  8.26390375e-02,
         8.85894646e-02, -5.60905533e-01, -1.06263640e-01,
        -9.56283756e-01, -1.11136421e+00, -5.16215243e-02,
        -1.52020390e-02, -6.51207483e-02,  9.37334487e-01,
        -9.81872727e-01,  1.54308479e-01,  1.23222986e+00,
        -1.06136229e-01, -2.83990988e-01,  1.06736598e+00,
        -2.93140432e-01,  7.88078276e-01,  8.76250372e-01,
         1.62359589e+00,  1.15954583e+00, -9.41201269e-0

         8.05060330e-03,  4.27301454e-01,  8.25994125e-01,
         3.68498627e-01, -6.61842756e-01,  4.35752258e-01,
         1.41583960e+00,  5.15927521e-01,  3.41648091e-01,
        -1.01723853e-01, -2.41927826e+00,  6.53708363e-01,
        -4.65547649e-01,  6.37575545e-01,  5.58926408e-01,
         3.13284876e-01, -1.52072709e-01,  3.96943623e-01,
         6.75978946e-02, -1.34085475e-01,  4.27848050e-01,
        -6.32202663e-01, -5.91339722e-01, -8.08684730e-01,
         4.43018332e-01, -1.41160484e+00, -6.61540220e-01,
         1.56789743e+00, -3.16217849e-01, -8.63784486e-01,
         3.58069465e-01, -1.54056167e-01, -8.09550611e-02,
        -4.71444180e-01, -1.33899792e-01, -2.79119485e-01,
        -3.52786531e-01, -6.68059057e-01, -8.96224180e-01,
         1.17381595e+00,  1.49857150e-01,  2.96109023e-02,
         5.50589627e-01, -4.26466175e-01, -1.44082262e-01,
         2.12731690e-01,  9.92610515e-01, -6.94550670e-01,
         3.55075363e-01,  8.64079465e-01, -3.79585908e-0

number white:  12172 density:  0.18572998046875
frame0009.png
number white:  12172 density:  0.18572998046875
frame0010.png
number white:  11516 density:  0.17572021484375
frame0011.png
number white:  11516 density:  0.17572021484375
frame0012.png
number white:  11516 density:  0.17572021484375
frame0013.png
number white:  12172 density:  0.18572998046875
frame0014.png
number white:  11516 density:  0.17572021484375
frame0015.png
number white:  11516 density:  0.17572021484375
frame0016.png
number white:  13996 density:  0.21356201171875
frame0017.png
number white:  13996 density:  0.21356201171875
frame0018.png
number white:  13996 density:  0.21356201171875
frame0019.png
number white:  13040 density:  0.198974609375
frame0020.png
number white:  12608 density:  0.1923828125
frame0021.png
number white:  12608 density:  0.1923828125
frame0022.png
number white:  12608 density:  0.1923828125
frame0023.png
number white:  12608 density:  0.1923828125
frame0024.png
number white:  12608 densi

number white:  12172 density:  0.18572998046875
frame0144.png
number white:  32808 density:  0.5006103515625
frame0145.png
number white:  32800 density:  0.50048828125
frame0146.png
number white:  31288 density:  0.4774169921875
frame0147.png
number white:  26816 density:  0.4091796875
frame0148.png
number white:  26816 density:  0.4091796875
frame0149.png
number white:  26816 density:  0.4091796875
frame0150.png
number white:  18512 density:  0.282470703125
frame0151.png
number white:  15608 density:  0.2381591796875
frame0152.png
number white:  13996 density:  0.21356201171875
frame0153.png
number white:  13996 density:  0.21356201171875
frame0154.png
number white:  13996 density:  0.21356201171875
frame0155.png
number white:  13040 density:  0.198974609375
frame0156.png
number white:  13040 density:  0.198974609375
frame0157.png
number white:  12612 density:  0.19244384765625
frame0158.png
number white:  12608 density:  0.1923828125
frame0159.png
number white:  12172 density:  0.185

## Image to STL File

In [311]:
import vox2STL

In [312]:
# first parameter: Folder with image stacks
vox1=vox2STL.img2vox('./output_leaf/img_to_architecture/'+label+'/stack/')

vox2STL.vox2stl(vox1, filename='output_leaf/STLs/img2arch_'+label)

found 1 images. ['./output_leaf/img_to_architecture/square/stack\\0255_smoothed.png']
save stl model to ./output_leaf/STLs/img2arch_square_12_03_13_51.stl


In [338]:
label='line'
path="./output_leaf/img_to_architecture/"+label+"/processed/"
dest="./output_leaf/img_to_architecture/"+label+"/"
stack_xy(path2 = path, dest=dest, ress=32, ressfinal=4096, thicken=2, rect=0, destfile=None)

Initial res:  32 final max res:  4096
Thickening resulting structure using dilation...iterations:  2
Largest res in final image:  512 ... but should be:  4096 ... ratio:  8.0
